<a href="https://colab.research.google.com/github/MMRES-PyBootcamp/MMRES-python-bootcamp/blob/master/08_scipy_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 8 - Statistics in Python (2<sup>nd</sup> part - 40')

TODO: Need more time than just 40'!

> A brief introduction on intermediate statistics topics in Python. Here we will remember the concept of *correlation*, showing some nice visualization strategies along the way (heat maps and dendrograms). We will also share some examples on how to implement *statistical hypothesis testing* with *SciPy*. Finally, we will try to show how to implement and interpret simple and multiple linear regressions with *StatsModels*.

## Outline
 * [Loading a dataset](#Loading-a-dataset) 
 * [Pearson correlation](#Pearson-correlation)
 * [Spearman correlation](#Spearman-correlation)
 * [Linear regression (simple)](#Linear-regression-(simple))
 * [Student's t-test](#Student's-t-test)
   * [One-sample t-test](#One-sample-t-test)
   * [Two-sample t-test (Independent samples t-test)](#Two-sample-t-test-(Independent-samples-t-test))
   * [Paired-samples t-test (Dependent samples t-test)](#Paired-samples-t-test-(Dependent-samples-t-test))
 * [Linear regression (multiple)](#Linear-regression-(multiple))

This document is devised as a tool to enable your **self-learning process**. If you get stuck at some step or need any kind of help, please don't hesitate to raise your hand and ask for the teacher's guidance. Along it, you will find some **special cells**:

<div class="alert alert-block alert-success"><b>Practice:</b> Practice cells announce exercises that you should try during the current boot camp session. Usually, solutions are provided using hidden cells (look for the dot dot dot symbol "..." and click to unravel them and check that your try is correct).
</div>

<div class="alert alert-block alert-warning"><b>Extension:</b> Extension cells correspond to exercises (or links to contents) that are a bit more advanced. We recommend to try them after the current boot camp session.
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> Tip cells just give some advice or complementary information.
</div>

<div class="alert alert-block alert-danger"><b>Caveat:</b> Caveat cells warn you about the most common pitfalls one founds when starts his/her path learning Python.

</div>

## Introduction

Since covering these topics in depth in such a reduced time is hard, we decided to approach it as a small demonstration built on short examples. To this aim, we took a [learning dataset](https://lectures.scientific-python.org/packages/statistics/index.html) from the SciPy website: the "IQ dataset". As we already said, the main Python packages we will leverage along this session are [SciPy](https://scipy.org/) and [StatsModels](https://www.statsmodels.org/stable/index.html).

## Loading a dataset

Let's start by loading all the packages we will need along the way and the [IQ dataset](https://lectures.scientific-python.org/packages/statistics/index.html)" from the SciPy website:

In [ ]:
# Load packages with their corresponding alias
import scipy as sp
import numpy as np
import pandas as pd

# Load plotting packages/classes with their corresponding alias
import plotnine as p9
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Loading the "IQ" dataset directly from an URL
df = pd.read_csv(
    filepath_or_buffer='https://scipy-lectures.org/_downloads/brain_size.csv',
    sep=';',
    na_values=".",
    index_col=0
)

# Show df's head
df.head()

The columns *FSIQ*, *VIQ* and *PIQ* stand for different "intelligence quotients" (in arbitrary units I guess) and the *MRI_Count* is another "intelligence" measure based on magnetic resonance imaging (also in some kind of arbitrary unit I guess). The columns *Gender*, *Weight* and *Height* express the gender, the weight (in pounds) and the height (in inches) of the individuals, respectively. Before starting, let's express *Weight* and *Height* column values to metric units to avoid an eventual [Mars probe crash](https://www.simscale.com/blog/nasa-mars-climate-orbiter-metric/).

In [ ]:
# Convert from imperial units to metric units
df['Weight [kg]'] = df['Weight'] * 0.45359237
df['Height [m]'] = df['Height'] * 0.0254

# Drop imperial columns
df.drop(columns=['Weight', 'Height'], inplace=True)

# Show df's head
df.head()

## Pearson correlation
We can compute the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) using the [`pearsonr()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html) function from *SciPy*'s `stats` module. This function takes two arrays and returns a tuple containing *Pearson correlation coefficient* and the *significance of the correlation* as p-value.

In [ ]:
# Get Pearson between 'FSIQ' and 'VIQ' using SciPy
sp.stats.pearsonr(x=df['FSIQ'], y=df['VIQ'])

We can also get the Pearson correlation coefficient using the [`.corr()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) **Pandas** method:

In [ ]:
# Get Pearson between 'FSIQ' and 'VIQ' using Pandas
df['FSIQ'].corr(df['VIQ'], method="pearson")

Look what happens when we apply this `.corr()` method to a Dataframe:

In [ ]:
# Get Pearson for all pair-wise pairs 'FSIQ' and 'VIQ' using Pandas
df[['FSIQ', 'VIQ']].corr(method='pearson')

By using the **Pandas** method `corr()` on a dataframe, we obtain all possible pair-wise correlations nicely displayed in a "square" dataframe (or matrix). Let's try this strategy again on the whole `df` dataframe (note the trick of the `numeric_only=` argument to avoid the error that pops up when trying to compute correlations involving the *Gender* column):

In [ ]:
# Get all pair-wise Pearson from our df
df_PCC = df.corr(method='pearson', numeric_only=True)

# Show the Pearson correlation matrix
df_PCC

Now let's leverage Seaborn [`.heatmap()`](https://seaborn.pydata.org/generated/seaborn.heatmap.html) to visualize this!

In [ ]:
# Plot the correlation matrix (HEATMAP!)
sns.heatmap(data=df_PCC, cmap="Blues", annot=True)

It seems that *FSIQ*, *VIQ* and *PIQ* are very correlated, and similarly, *Weight [kg]*, *Height [m]* and (to a lesser extend) *MRI_Count*.

## Spearman correlation

Pearson correlation assumes that the data we are comparing is <u>**normally distributed**</u>. When this assumption is not true, the Pearson correlation value is not reflecting the true association. Spearman correlation does not assume that data follows a specific distribution, so it is a non-parametric correlation measure. Spearman correlation is also known as Spearman's rank correlation as it computes correlation coefficient on rank values of the data. Using *SciPy*, we can compute Spearman correlation using the function `spearmanr()`:

In [ ]:
#The first element of tuple is the Spearman's rank correlation and the second is p-value.
sp.stats.spearmanr(a=df['FSIQ'], b=df['VIQ'])

<div class="alert alert-block alert-info">

<b> Tip:</b> I don't know why `sp.stats.pearsonr()` uses `x=` and `y=` as argument names and `sp.stats.spearmanr()` uses `a=` and `b=`. In any case, you can use these two functions without specifying argument names: `sp.stats.pearsonr(df['FSIQ'], df['VIQ'])` or `sp.stats.spearmanr(df['FSIQ'], df['VIQ'])`.

</div>

Again, we can also get the Spearman correlation coefficient using the [`.corr()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) **Pandas** method:

In [ ]:
# Get Spearman between 'FSIQ' and 'VIQ' using Pandas
df['FSIQ'].corr(df['VIQ'], method="spearman")

Now, we will use the Seaborn function [`clustermap()`](https://seaborn.pydata.org/generated/seaborn.clustermap.html) instead of `heatmap()`:

In [ ]:
# Get all pair-wise Spearman from our df
df_ECC = df.corr(method='spearman', numeric_only=True)

# Show the Spearman correlation matrix
df_ECC

# Plot the Spearman correlation matrix (CLUSTERMAP!)
sns.clustermap(data=df_ECC, cmap="Reds", annot=True, z_score=None, figsize=(6,6))

Have a look at the [dendrogram](https://en.wikipedia.org/wiki/Dendrogram) and note how it hierarchizes closer those variables more correlated.

## Linear regression (simple)

With the *SciPy* function [`linregress()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html) it's very easy to perform a *simple linear regression* and get all required output coefficients.

In [ ]:
# Compute a linear regression between 'FSIQ' and 'VIQ' using SciPy
sp.stats.linregress(x=df['FSIQ'], y=df['VIQ'])

Note how `sp.stats.linregress()` returns an object storing five values: `slope`, `intercept`, `rvalue`, `pvalue`, `stderr` and `intercept_stde`. You can assign the output of `sp.stats.linregress()` in a variable and easily access its elements as follows:

In [ ]:
# Compute a linear regression between 'FSIQ' and 'VIQ' using SciPy
FitResults = sp.stats.linregress(x=df['FSIQ'], y=df['VIQ'])

# Return the 'rvalue' (Pearson correlation) generated by the linear regression
FitResults.rvalue

By using some f-string trickery we can easily prepare a nice annotation with the regression line equation:

In [ ]:
# Gather the regression line equation (using just 3 decimal places with the :.3f trick)
annotation = f'y(x) = {FitResults.slope:.3f} x + {FitResults.intercept:.3f}'

# Show the regression line equation
print(annotation)

Now, let's use *Seaborn* to visualize the regression line along with the data:

In [ ]:
# Create the scatter plot with Seaborn
p = sns.scatterplot(data=df, x="FSIQ", y="VIQ", hue="Gender")

# Add the linear regression line (by hand) with Matplotlib
p.axline(xy1=(0, FitResults.intercept), slope=FitResults.slope, color='lightgray')

# Adjust scatter plot limits (by hand) with Matplotlib
p.set_xlim(df["FSIQ"].min()-2, df["FSIQ"].max()+2)
p.set_ylim(df["VIQ"].min()-2, df["VIQ"].max()+2) 

# Add regression line equation (by hand) with Matplotlib
plt.text(95, 145, annotation)

With *Plotnine* we can also obtain a very nice plot (maybe with less tweaking):

TODO: Pre-install `plotnine` if we are going to use it!

In [ ]:
gg = (
    
    # Create the scatter plot with Plotnine
    p9.ggplot(data=df, mapping=p9.aes(x="FSIQ", y="VIQ", color="Gender"))
     + p9.geom_point()

    # Add the linear regression line (by hand)
     + p9.geom_abline(intercept=FitResults.intercept, slope=FitResults.slope, color='lightgray')

    # Add regression line equation (by hand)
     + p9.annotate(geom="text", x=95, y=145, label=annotation)
)

# Show plot
gg

## Student's t-test

A t-test is a statistical tool used to determine if there is a significant difference between the means of two groups, or between the mean of one group and a known value. There are three kinds of t-tests: 
 * One-Sample t-test
 * Two-sample t-test
 * Paired samples t-test

### One-sample t-test

The one-sample t-test is used to <u>**compare the mean of a single group to a known or hypothesized value**</u>. This test is appropriate when you want to determine if a sample comes from a population with a specific mean. *SciPy* has implemented a function to perform one-sample t-test: [`sp.stats.ttest_1samp()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html). Let's see an example:

In [ ]:
# Represent the distribution traced by VIQ and mark its mean
sns.kdeplot(data=df, x="VIQ", color='blue', fill='blue', alpha=0.1)
plt.axvline(x=df['VIQ'].mean(), color='blue')

# Mark a couple of arbitrary values
plt.axvline(x=100, color='red')
plt.axvline(x=105, color='green')

To test if the difference between the mean of *VIQ* and $105$ is significant, we should do:

In [ ]:
# Launching a 1STT to check if VIQ mean differs than 105
sp.stats.ttest_1samp(a=df['VIQ'], popmean=105)

This function returns an object with the *T statistic*, the *p-value* and the *degrees of freedom*. If, for example, we assume a significance level of $0.05$, we cannot say that the mean of *VIQ* is different than $105$. In contrast, with this same significance level, we can say that the mean of *VIQ* is different than $100$:

In [ ]:
# Launching a 1STT to check if VIQ mean differs than 100
sp.stats.ttest_1samp(a=df['VIQ'], popmean=100)

### Two-sample t-test (Independent samples t-test)

The two-sample t-test is used to <u>**compare the means of two unrelated groups**</u>. The term "unrelated" means that the two groups are not related to each other (different patients with and without a given clinical condition for example). *SciPy* has implemented a function to perform two-sample t-test: [`sp.stats.ttest_ind()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html). For example, let's compare the mean *VIQ* between females and males:

In [ ]:
# Group by 'Gender' and get the mean VIQ
df_g = df.groupby(by='Gender').agg(Mean_VIQ=('VIQ', 'mean'))

# Show the data frame
df_g

In [ ]:
# Represent the distribution traced by VIQ differentiating males and females
sns.kdeplot(data=df, x="VIQ", hue='Gender', fill='Gender')

# Mark the mean VIQ for males (orange) and females (blue)
plt.axvline(x=df_g['Mean_VIQ']['Male'], color=sns.color_palette()[1])
plt.axvline(x=df_g['Mean_VIQ']['Female'], color=sns.color_palette()[0])
# Note how we used the same default colors used by Seaborn with sns.color_palette()[ ] ;)

It seems that the mean *VIQ* could be slightly different between genders, but is this difference *statistically significant*? Let's check it out!

In [ ]:
# Prepare filters for females and males
filter_F = df['Gender'] == 'Female'
filter_M = df['Gender'] == 'Male'

# Launching a 2STT to check if VIQ mean differs between females and males
sp.stats.ttest_ind(df[filter_F]['VIQ'], df[filter_M]['VIQ'])

Assuming again the typical significance level of $0.05$, we cannot report a significant *VIQ* different between genders.

### Paired-samples t-test (Dependent samples t-test)

Finally, the paired samples t-test is used to <u>**compare the means of two related groups**</u>. Here, the term "related" denotes that the groups are not independent (same patients before and after a treatment for example). *SciPy* implements a function to perform paired-samples t-test: [`sp.stats.ttest_rel()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html). In the previous example we compared *VIQ* between females and males (independent groups). Now, let's compare the three IQ estimators provided in our data set (*PIQ*, *VIQ*, and *FSIQ*).

<div class="alert alert-block alert-info">

<b>Tip:</b> We want to visualize the three IQ estimators in a same plot to better glimpse whats going on. In order to easily do that leveraging Seaborn, we first need to tweak a bit the original data frame `df` by melting it. By the way, we will also get the mean for each IQ estimator as we just did to get the mean <i>VIQ</i> for females and males.

</div>

In [ ]:
# Melting our data frame to better visualize it with Seaborn
df_m = pd.melt(frame=df, id_vars=None, value_vars=['PIQ', 'VIQ', 'FSIQ'], var_name='IQ type', value_name='IQ value') 

# Show the data frame
df_m

In [ ]:
# Group by 'IQ type' and get the mean IQ
df_g = df_m.groupby(by='IQ type').agg(Mean_IQ=('IQ value', 'mean'))

# Show the data frame
df_g

In [ ]:
# Represent the distribution traced by each IQ type
sns.kdeplot(data=df_m, x="IQ value", hue='IQ type', fill='IQ type')

# Mark the mean IQ for each IQ type: PIQ (blue), VIQ (orange), FSIQ (green)
plt.axvline(x=df_g['Mean_IQ']['PIQ'], color=sns.color_palette()[0])
plt.axvline(x=df_g['Mean_IQ']['VIQ'], color=sns.color_palette()[1])
plt.axvline(x=df_g['Mean_IQ']['FSIQ'], color=sns.color_palette()[2])
# Note how we used the same default colors used by Seaborn with sns.color_palette()[ ] ;)

At first sight, all three IQ estimators exhibit quite alike bimodal distributions with very similar means for `IQ value`. Now, let's check it out!

In [ ]:
# Launching a PSTT to check if PIQ mean differs from VIQ mean
print(sp.stats.ttest_rel(df['PIQ'], df['VIQ']))

# Launching a PSTT to check if PIQ mean differs from FSIQ mean
print(sp.stats.ttest_rel(df['PIQ'], df['FSIQ']))

# Launching a PSTT to check if VIQ mean differs from FSIQ mean
print(sp.stats.ttest_rel(df['VIQ'], df['FSIQ']))

Assuming again a significance level of $0.05$, we cannot report a significant difference between the three IQ estimators available.

<div class="alert alert-block alert-info">
    
<b>Tip:</b> Imagine that (instead of three IQ estimators) you have, let's say, one hundred. Getting all possible pairs and writing the corresponding statistical tests by hand could be dangerous. In these situations you should <i>loop</i> along all possible pair combinations:

</div>

In [ ]:
# Load the itertools package
import itertools

# Define a list with all our IQ estimators
list_IQs = ['PIQ', 'VIQ', 'FSIQ']

# Get a list with all possible list_IQs combinations pairs (repeat=2)
list_IQ_pairs = itertools.combinations(iterable=list_IQs, r=2) 

# Loop along all possible pairs...
for pair in list_IQ_pairs:
    
    #... print the running pair and its corresponding paired samples t-test outcome
    print(pair, sp.stats.ttest_rel(df[pair[0]], df[pair[1]]), '\n')

<div class="alert alert-block alert-warning">
    
<b>Extension:</b> You can also achieve this with a *list comprehension* instead of a *for loop*:

</div>

In [ ]:
# Define a list with all our IQ estimators
list_IQs = ['PIQ', 'VIQ', 'FSIQ']

# Get a list with all possible list_IQs combinations pairs (repeat=2)
list_IQ_pairs = itertools.combinations(iterable=list_IQs, r=2) 

# Give a tuple ( , ) with the running pair and its corresponding paired samples t-test outcome for all possible pairs
[( pair, sp.stats.ttest_rel(df[pair[0]], df[pair[1]]) ) for pair in list_IQ_pairs]

<div class="alert alert-block alert-success">
    
<b>Practice 1:</b> Test if the weight for females and males is different using a t-test.

1) Which t-test will you use (one-, two- or paired-sample t-test)? Why?
2) In the 1<sup>st</sup> code cell below, use the chosen t-test to statistically evaluate the weight difference between females and males. **What happened in this first try?**
3) In the 2<sup>nd</sup> code cell below, try again with the `sp.stats.ttest_***()` argument `nan_policy='omit'`. **What happened in this second try?** What was the outcome of your t-test?
4) In the 3<sup>rd</sup> code cell below, prepare a nice plot showing the weight distribution for females and males, marking their corresponding mean values with vertical lines.

<b>Note:</b> Uncomment and fill only those code lines with underscores `___`.

</div>

In [ ]:
# (1st code cell)

# Prepare filters for females and males
#filter_F = ___
#filter_M = ___

# Launching a 2STT to check if the weight mean differs between females and males
#sp.stats.___(___, ___)

In [ ]:
# (1st code cell --> SOLUTION)

# Prepare filters for females and males
filter_F = df['Gender'] == 'Female'
filter_M = df['Gender'] == 'Male'

# Launching a 2STT to check if the weight mean differs between females and males
sp.stats.ttest_ind(df[filter_F]['Weight [kg]'], df[filter_M]['Weight [kg]'])

In [ ]:
# (2nd code cell)

# Launching a 2STT to check if the weight mean differs between females and males
#sp.stats.___(___, ___, ___)

In [ ]:
# (2nd code cell --> SOLUTION)

# Launching a 2STT to check if the weight mean differs between females and males
sp.stats.ttest_ind(df[filter_F]['Weight [kg]'], df[filter_M]['Weight [kg]'], nan_policy='omit')

In [ ]:
# (3rd code cell)

# Group by 'Gender' and get the mean weight
#df_g = df.groupby(by=___).agg(Mean_Weight_kg=(___, ___))

# Show the data frame
#print(df_g)

# Represent the distribution traced by weight differentiating males and females
#sns.kdeplot(data=___, x=___, hue=___, fill=___)

# Mark the mean weight for males (orange) and females (blue)
#plt.axvline(x=___['Mean_Weight_kg'][___], color=sns.color_palette()[___])
#plt.axvline(x=___['Mean_Weight_kg'][___], color=sns.color_palette()[___])
# Note how we used the same default colors used by Seaborn with sns.color_palette()[ ] ;)

In [ ]:
# (3rd code cell --> SOLUTION)

# Group by 'Gender' and get the mean weight
df_g = df.groupby(by='Gender').agg(Mean_Weight_kg=('Weight [kg]', 'mean'))

# Show the data frame
print(df_g)

# Represent the distribution traced by weight differentiating males and females
sns.kdeplot(data=df, x='Weight [kg]', hue='Gender', fill='Gender')

# Mark the mean weight for males (orange) and females (blue)
plt.axvline(x=df_g['Mean_Weight_kg']['Female'], color=sns.color_palette()[0])
plt.axvline(x=df_g['Mean_Weight_kg']['Male'], color=sns.color_palette()[1])
# Note how we used the same default colors used by Seaborn with sns.color_palette()[ ] ;)

<div class="alert alert-block alert-success"><b>Practice 1 ends here.</b>

</div>

## Linear regression (multiple)

To start with the *multiple linear regressions*, let's repeat the *simple linear regression* between *FSIQ* and *VIQ* we already performed but using the approach to perform multiple linear regressions. To this aim, we will need to import the `formula.api` from `statsmodels` so we can use the [R-style formula syntax](https://www.statsmodels.org/stable/example_formulas.html).

In [ ]:
# Load package/class with their corresponding alias
import statsmodels.formula.api as smf

# Create the linear model with the R-style formula syntax 
mod = smf.ols(formula="VIQ ~ FSIQ", data=df)

# Get the result after fitting the model 
res = mod.fit()

# Show the summary of the result obtained after fitting the model
print(res.summary())

Note that we recover the same coefficients obtained with `sp.stats.linregress(x=df['FSIQ'], y=df['VIQ'])`:

In [ ]:
# Compute a linear regression between 'FSIQ' and 'VIQ' using SciPy
FitResults = sp.stats.linregress(x=df['FSIQ'], y=df['VIQ'])

# Return the 'intercept' generated by the linear regression
print(f'Intecept: {FitResults.intercept}', sep='\n')

# Return the 'slope' generated by the linear regression
print(f'Slope: {FitResults.slope}')

<div class="alert alert-block alert-warning">
    
<b>Extension:</b> The R-style formula syntax is a commonly used framework to define statistical models. The *StatsModels* package internally uses the [*Patsy*](https://patsy.readthedocs.io/en/latest/#) package to convert formulas and data to the kind of matrices that are used in model fitting. This is a quite powerful and complex framework that requires a bit of practice.

</div>

Now let's add an extra dependent variable (*PIQ*) to our model:

In [ ]:
# Create the linear model with the R-style formula syntax 
mod = smf.ols(formula="VIQ ~ FSIQ + PIQ", data=df)

# Get the result after fitting the model 
res = mod.fit()

# Show the summary of the result obtained after fitting the model
print(res.summary())

As you can see, the summary of the model fitting result obtained gives a lot of data. Let's break down this info for this particular example:

* **OLS Regression Results**: The most important items of the first section in the summary are the `R-squared`, `F-statistic` and `Prob (F-statistic)`. In this particular example, the model is a very good fit for the data. The `R-squared` value of `0.985` indicates that approximately 98.5% of the variance in the dependent variable, *VIQ*, can be explained by the independent variables, *FSIQ* and *PIQ*. The overall model is statistically significant, as shown by the highly significant `F-statistic`, with a `Prob (F-statistic)` around `3.12e-34` (well below 0.05).

* **Independent variables coefficients**: The regression equation can be written as $VIQ (FSIQ, PIQ)= 17.66 + 1.69⋅FSIQ − 0.88⋅PIQ$ and the intuitive interpretation of the coeficients would be:
   * **FSIQ coefficient** (1.69): Holding the *PIQ* variable constant, a one-unit increase in *FSIQ* is associated with an average increase of 1.69 units in *VIQ*. This relationship is highly significant because the `t` statistic has an associated probability, `P>|t|`, well below 0.05.
     
   * **PIQ coefficient** (-0.88): Holding the *FSIQ* variable constant, a one-unit increase in *PIQ* is associated with an average decrease of 0.88 units in *VIQ*. Again, this relationship is highly significant because the `t` statistic has an associated probability, `P>|t|`, well below 0.05.
 
   * **Intercept** (17.66): The intercept represents the predicted value of *VIQ* when both *FSIQ* and *PIQ* are zero. While statistically significant, this value may not have a meaningful real-world interpretation, as a *VIQ* score of zero on these tests is very unlikely.

<div class="alert alert-block alert-success">
    
<b>Practice 2:</b>

Devise a model to explain any *IQ estimator* by means of the *anatomical data*.

1) In the 1<sup>st</sup> code cell below, implement a model to check if we can explain any of the three *IQ estimators* (*FSIQ*, *PIQ* or *VIQ*), using the weight and height. **What happened?**
2) In the 2<sup>nd</sup> code cell below, try again but renaming `df` columns `'Weight [kg]'` and `'Height [m]'` so *their names don't contain white spaces*. **What happened now?**
3) In the 3<sup>rd</sup> code cell below, fit your model, print the corresponding result summary, and have a detailed look at it (take your time). **What do you conclude?**

<b>Note:</b> Uncomment and fill only those code lines with underscores `___`.

</div>

In [ ]:
# (1st code cell)

# Create the linear model with the R-style formula syntax 
# mod = ___.___(formula="___ ~ ___ + ___", data=___)

In [ ]:
# (1st code cell --> SOLUTION)

# Create the linear model with the R-style formula syntax 
mod = smf.ols(formula="VIQ ~ Height [m] + Weight [kg]", data=df)

In [ ]:
# (2nd code cell)

# Rename df column so their names don't contain white spaces
dict_rename = {'Weight [kg]': 'Weight_kg', 'Height [m]': 'Height_m'}
df.rename(columns=dict_rename, inplace=True)

# Create the linear model with the R-style formula syntax 
# mod = ___.___(formula="___ ~ ___ + ___", data=df)

In [ ]:
# (2nd code cell --> SOLUTION)

# Rename df column so their names don't contain white spaces
dict_rename = {'Weight [kg]': 'Weight_kg', 'Height [m]': 'Height_m'}
df.rename(columns=dict_rename, inplace=True)

# Create the linear model with the R-style formula syntax 
mod = smf.ols(formula="VIQ ~ Height_m + Weight_kg", data=df)

In [ ]:
# (3rd code cell)

# Get the result after fitting the model 
#res = ___.fit()

# Show the summary of the result obtained after fitting the model
#print(___.summary())

In [ ]:
# (3rd code cell --> SOLUTION)

# Get the result after fitting the model 
res = mod.fit()

# Show the summary of the result obtained after fitting the model
print(res.summary())

<div class="alert alert-block alert-success">

<b> Practice 2 ends here.</b>

</div>

Finally, multiple linear regressions can also work with categorical variables. For example:

In [ ]:
# Create the linear model with the R-style formula syntax 
mod = smf.ols(formula="VIQ ~ Gender -1", data=df)

# Get the result after fitting the model 
res = mod.fit()

# Show the summary of the result obtained after fitting the model
print(res.summary())

This regression model was intended to predict a person's *VIQ* based on its *Gender*. However, is a very poor fit for the data. Let's see it in more detail!

* **Independent variables coefficients**: The model suggests that, on average, males and females have similar *VIQ* values. In particular:

  * **Gender[Female] coefficient**: The average *VIQ* for females in this dataset is `109.45`, with an `std err` (uncertainty) of `5.308`.

  * **Gender[Male] coefficient**: The average *VIQ* for males in this dataset is `115.25`, also with an `std err` of `5.308`.

* **OLS Regression Results**: Despite the individual coefficients being statistically significant, the model as a whole is not useful for prediction. The `R-squared` of `0.015` indicates that only 1.5% of the variability in *VIQ* is explained by gender. The `F-statistic` is `0.5969` with a `Prob (F-statistic)` of `0.445`, which is not statistically significant. This means the model does not provide a better fit than a model with no predictors.

<div class="alert alert-block alert-warning">
    
<b>Extension:</b> Did you noticed the `-1` at the end of the formula? We used this trick to *remove the intercept* from the fitting formula, which means that the fitting coefficients `Gender[Female]` and `Gender[Male]` represent the average *VIQ* by *Gender*.

</div>

As you can see, implementing statistical analyses in Python is easy. The hardest thing is to truly know what you are doing. For example: *this* test can be applied to *this* data? I'm correctly interpreting the output of this *linear regression*? Is my statistical model properly devised? etc...